## Missing Data, Compositional Data, Minimum Dimensionality and Classifier Accuracy

**Data** [Sources](2_Data_Sources.ipynb) | [Compositional Data](2_Data_CompositionalData.ipynb) | [Lambdas](2_Data_Lambdas.ipynb) **Classifiers** [Models](3_Classifiers_ClassifierModels.ipynb) | [Dimensionality](3_Classifiers_Dimensionality.ipynb) | [Performance](3_Classifiers_Performance.ipynb) **Visualisation**  [Entropy](4_Vis_Entropy.ipynb) | [Manifolds](4_Vis_Manifolds.ipynb) **Workflows** [Building Workflows](5_BuildingWorkflows.ipynb) **pyrolite** [Overview](6_pyrolite.ipynb) **GitHub** [AEGC2019](https://github.com/morganjwilliams/aegc2019) | [pyrolite]

* The performance of classifiers is typically closely linked to their dimensionality.
* With compositional data, missing data can be a central limitation to using higher dimensional datasets.
* Accessing dense high-dimensional datasets can be difficult, and carefully choosing a subset of variables is often the best course of action.

### Discrimination Power of Multivariate Data

### Understanding Your Dataset

#### Correlation and Co-occurrence